In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_columns', None)

In [105]:
df = pd.read_csv('data/T_T100_SEGMENT_ALL_CARRIER.csv')
area_codes_table = pd.read_csv('data/L_WORLD_AREA_CODES.csv')
carrier_classes = pd.read_csv('data/L_CARRIER_GROUP_NEW.csv')

In [103]:
carrier_classes

,Code,Description,Carrier Class
0,0,Foreign Carriers,Foreign
1,1,Large Regional Carriers (carriers with annual ...,Large Regional
2,2,National Carriers (carriers with annual revenu...,National
3,3,Major Carriers (carriers with annual revenue o...,Major
4,4,Medium Regional Carriers (carriers with annual...,Medium Regional
5,5,Small Certificated Carriers (carrier holding c...,Small Certificated
6,6,Commuter Carriers (air taxi operator which per...,Commuter
7,7,All Cargo Carriers operating under cerificates...,All Cargo
8,9,Commuter Carriers (Air Taxi providing Essentia...,Commuter


In [40]:
# Replace World Area Codes (WAC) with descriptions from lookup table
df = df.merge(area_codes_table, left_on='ORIGIN_WAC', right_on='Code', how='left')
df['ORIGIN_WAC'] = df['Description']
df = df.drop(columns=['Code', 'Description'])
df = df.merge(area_codes_table, left_on='DEST_WAC', right_on='Code', how='left')
df['DEST_WAC'] = df['Description']
df = df.drop(columns=['Code', 'Description'])

In [89]:
# Create simplified list of carrier class descriptions
carrier_classes_modified = []
for carrier_type in carrier_classes['Description']:
    index = carrier_type.index('Carrier')
    substr = carrier_type[:index-1]
    carrier_classes_modified.append(substr)

carrier_classes['Carrier Class'] = carrier_classes_modified
carrier_classes

,Code,Description,Carrier Class
0,0,Foreign Carriers,Foreign
1,1,Large Regional Carriers (carriers with annual ...,Large Regional
2,2,National Carriers (carriers with annual revenu...,National
3,3,Major Carriers (carriers with annual revenue o...,Major
4,4,Medium Regional Carriers (carriers with annual...,Medium Regional
5,5,Small Certificated Carriers (carrier holding c...,Small Certificated
6,6,Commuter Carriers (air taxi operator which per...,Commuter
7,7,All Cargo Carriers operating under cerificates...,All Cargo
8,9,Commuter Carriers (Air Taxi providing Essentia...,Commuter


In [106]:
# Replace carrier groups with descriptions from modified lookup table

#Convert from int to prevent type mismatch
#df['CARRIER_GROUP_NEW'] = df['CARRIER_GROUP_NEW'].astype(str)
df = df.merge(carrier_classes, left_on='CARRIER_GROUP_NEW', right_on='Code', how='left')
df

,DEPARTURES_SCHEDULED,DEPARTURES_PERFORMED,PAYLOAD,SEATS,PASSENGERS,FREIGHT,MAIL,DISTANCE,RAMP_TO_RAMP,AIR_TIME,UNIQUE_CARRIER,UNIQUE_CARRIER_NAME,REGION,CARRIER,CARRIER_NAME,CARRIER_GROUP_NEW,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_NM,ORIGIN_COUNTRY_NAME,ORIGIN_WAC,DEST_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEST_STATE_NM,DEST_COUNTRY_NAME,DEST_WAC,AIRCRAFT_GROUP,AIRCRAFT_TYPE,AIRCRAFT_CONFIG,YEAR,QUARTER,MONTH,DISTANCE_GROUP,CLASS,DATA_SOURCE,Code,Description
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,NEW,New England Airlines Inc.,D,NEW,New England Airlines Inc.,6,30613,BID,"Block Island, RI",Rhode Island,United States,15,35855,WST,"Westerly, RI",Rhode Island,United States,15,0,79,1,2024,1,1,1,F,DU,6,Commuter Carriers (air taxi operator which per...
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,NEW,New England Airlines Inc.,D,NEW,New England Airlines Inc.,6,30613,BID,"Block Island, RI",Rhode Island,United States,15,35855,WST,"Westerly, RI",Rhode Island,United States,15,0,79,1,2024,3,7,1,F,DU,6,Commuter Carriers (air taxi operator which per...
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,NEW,New England Airlines Inc.,D,NEW,New England Airlines Inc.,6,35855,WST,"Westerly, RI",Rhode Island,United States,15,30613,BID,"Block Island, RI",Rhode Island,United States,15,0,79,1,2024,1,1,1,F,DU,6,Commuter Carriers (air taxi operator which per...
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,NEW,New England Airlines Inc.,D,NEW,New England Airlines Inc.,6,35855,WST,"Westerly, RI",Rhode Island,United States,15,30613,BID,"Block Island, RI",Rhode Island,United States,15,0,79,1,2024,3,7,1,F,DU,6,Commuter Carriers (air taxi operator which per...
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.0,0.0,0.0,9K,Cape Air,D,9K,Cape Air,6,33541,MVY,"Martha's Vineyard, MA",Massachusetts,United States,13,32250,HYA,"Hyannis, MA",Massachusetts,United States,13,1,125,1,2024,1,1,1,F,DU,6,Commuter Carriers (air taxi operator which per...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
314502,891.0,889.0,2367000.0,8002.0,4689.0,5250.0,0.0,59.0,33895.0,26497.0,GV,Grant Aviation,D,GV,Grant Aviation,5,31555,ENA,"Kenai, AK",Alaska,United States,1,30299,ANC,"Anchorage, AK",Alaska,United States,1,4,416,1,2024,2,6,1,F,DU,5,Small Certificated Carriers (carrier holding c...
314503,995.0,994.0,2737200.0,8946.0,7247.0,10718.0,0.0,59.0,37921.0,27463.0,GV,Grant Aviation,D,GV,Grant Aviation,5,30299,ANC,"Anchorage, AK",Alaska,United States,1,31555,ENA,"Kenai, AK",Alaska,United States,1,4,416,1,2024,3,7,1,F,DU,5,Small Certificated Carriers (carrier holding c...
314504,999.0,998.0,2748200.0,8991.0,6631.0,5663.0,0.0,59.0,39218.0,30137.0,GV,Grant Aviation,D,GV,Grant Aviation,5,31555,ENA,"Kenai, AK",Alaska,United States,1,30299,ANC,"Anchorage, AK",Alaska,United States,1,4,416,1,2024,3,7,1,F,DU,5,Small Certificated Carriers (carrier holding c...
314505,1522.0,1522.0,1826400.0,8026.0,5084.0,10340.0,0.0,11.0,17768.0,13202.0,S2,Star Marianas Air Inc.,D,S2,Star Marianas Air Inc.,6,35215,TIQ,"Tinian, TT",U.S. Pacific Trust Territories and Possessions,United States,5,34955,SPN,"Saipan, TT",U.S. Pacific Trust Territories and Possessions,United States,5,0,79,1,2024,3,7,1,F,DU,6,Commuter Carriers (air taxi operator which per...


In [41]:
df[:10]

,DEPARTURES_SCHEDULED,DEPARTURES_PERFORMED,PAYLOAD,SEATS,PASSENGERS,FREIGHT,MAIL,DISTANCE,RAMP_TO_RAMP,AIR_TIME,UNIQUE_CARRIER,UNIQUE_CARRIER_NAME,REGION,CARRIER,CARRIER_NAME,CARRIER_GROUP_NEW,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_NM,ORIGIN_COUNTRY_NAME,ORIGIN_WAC,DEST_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEST_STATE_NM,DEST_COUNTRY_NAME,DEST_WAC,AIRCRAFT_GROUP,AIRCRAFT_TYPE,AIRCRAFT_CONFIG,YEAR,QUARTER,MONTH,DISTANCE_GROUP,CLASS,DATA_SOURCE
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,NEW,New England Airlines Inc.,D,NEW,New England Airlines Inc.,6,30613,BID,"Block Island, RI",Rhode Island,United States,Rhode Island,35855,WST,"Westerly, RI",Rhode Island,United States,Rhode Island,0,79,1,2024,1,1,1,F,DU
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,NEW,New England Airlines Inc.,D,NEW,New England Airlines Inc.,6,30613,BID,"Block Island, RI",Rhode Island,United States,Rhode Island,35855,WST,"Westerly, RI",Rhode Island,United States,Rhode Island,0,79,1,2024,3,7,1,F,DU
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,NEW,New England Airlines Inc.,D,NEW,New England Airlines Inc.,6,35855,WST,"Westerly, RI",Rhode Island,United States,Rhode Island,30613,BID,"Block Island, RI",Rhode Island,United States,Rhode Island,0,79,1,2024,1,1,1,F,DU
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,NEW,New England Airlines Inc.,D,NEW,New England Airlines Inc.,6,35855,WST,"Westerly, RI",Rhode Island,United States,Rhode Island,30613,BID,"Block Island, RI",Rhode Island,United States,Rhode Island,0,79,1,2024,3,7,1,F,DU
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.0,0.0,0.0,9K,Cape Air,D,9K,Cape Air,6,33541,MVY,"Martha's Vineyard, MA",Massachusetts,United States,Massachusetts,32250,HYA,"Hyannis, MA",Massachusetts,United States,Massachusetts,1,125,1,2024,1,1,1,F,DU
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,0.0,0.0,9K,Cape Air,D,9K,Cape Air,6,33541,MVY,"Martha's Vineyard, MA",Massachusetts,United States,Massachusetts,30154,ACK,"Nantucket, MA",Massachusetts,United States,Massachusetts,1,125,1,2024,1,2,1,F,DU
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,0.0,0.0,DL,Delta Air Lines Inc.,D,DL,Delta Air Lines Inc.,3,31703,HPN,"White Plains, NY",New York,United States,New York,31703,JFK,"New York, NY",New York,United States,New York,6,622,1,2024,1,2,1,L,DU
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49.0,0.0,0.0,DL,Delta Air Lines Inc.,D,DL,Delta Air Lines Inc.,3,30721,BOS,"Boston, MA",Massachusetts,United States,Massachusetts,30721,PVD,"Providence, RI",Rhode Island,United States,Rhode Island,6,622,1,2024,3,7,1,L,DU
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,54.0,0.0,0.0,9K,Cape Air,D,9K,Cape Air,6,34992,STX,"Christiansted, VI",U.S. Virgin Islands,United States,U.S. Virgin Islands,35626,VQS,"Vieques, PR",Puerto Rico,United States,Puerto Rico,1,125,1,2024,2,4,1,F,DU
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80.0,0.0,0.0,DL,Delta Air Lines Inc.,D,DL,Delta Air Lines Inc.,3,33342,MKE,"Milwaukee, WI",Wisconsin,United States,Wisconsin,30977,MDW,"Chicago, IL",Illinois,United States,Illinois,6,622,1,2024,2,4,1,L,DU


In [17]:
df['DEST_WAC'] = df['Description']
df['ORIGIN_WAC'] = df['Description']
df = df.drop(columns=['Code', 'Description'])

KeyError: 'Description'

In [14]:
df['Load Factor'] = df['PASSENGERS'] / df['SEATS']
df['Load Factor'] = df['Load Factor'].fillna(0)

In [16]:
df[:10]

,DEPARTURES_SCHEDULED,DEPARTURES_PERFORMED,PAYLOAD,SEATS,PASSENGERS,FREIGHT,MAIL,DISTANCE,RAMP_TO_RAMP,AIR_TIME,UNIQUE_CARRIER,UNIQUE_CARRIER_NAME,REGION,CARRIER,CARRIER_NAME,CARRIER_GROUP_NEW,ORIGIN_CITY_MARKET_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_NM,ORIGIN_COUNTRY_NAME,ORIGIN_WAC,DEST_CITY_MARKET_ID,DEST,DEST_CITY_NAME,DEST_STATE_NM,DEST_COUNTRY_NAME,DEST_WAC,AIRCRAFT_GROUP,AIRCRAFT_TYPE,AIRCRAFT_CONFIG,YEAR,QUARTER,MONTH,DISTANCE_GROUP,CLASS,DATA_SOURCE,Load Factor
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,NEW,New England Airlines Inc.,D,NEW,New England Airlines Inc.,6,30613,BID,"Block Island, RI",Rhode Island,United States,Rhode Island,35855,WST,"Westerly, RI",Rhode Island,United States,Rhode Island,0,79,1,2024,1,1,1,F,DU,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,NEW,New England Airlines Inc.,D,NEW,New England Airlines Inc.,6,30613,BID,"Block Island, RI",Rhode Island,United States,Rhode Island,35855,WST,"Westerly, RI",Rhode Island,United States,Rhode Island,0,79,1,2024,3,7,1,F,DU,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,NEW,New England Airlines Inc.,D,NEW,New England Airlines Inc.,6,35855,WST,"Westerly, RI",Rhode Island,United States,Rhode Island,30613,BID,"Block Island, RI",Rhode Island,United States,Rhode Island,0,79,1,2024,1,1,1,F,DU,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,NEW,New England Airlines Inc.,D,NEW,New England Airlines Inc.,6,35855,WST,"Westerly, RI",Rhode Island,United States,Rhode Island,30613,BID,"Block Island, RI",Rhode Island,United States,Rhode Island,0,79,1,2024,3,7,1,F,DU,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.0,0.0,0.0,9K,Cape Air,D,9K,Cape Air,6,33541,MVY,"Martha's Vineyard, MA",Massachusetts,United States,Massachusetts,32250,HYA,"Hyannis, MA",Massachusetts,United States,Massachusetts,1,125,1,2024,1,1,1,F,DU,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,0.0,0.0,9K,Cape Air,D,9K,Cape Air,6,33541,MVY,"Martha's Vineyard, MA",Massachusetts,United States,Massachusetts,30154,ACK,"Nantucket, MA",Massachusetts,United States,Massachusetts,1,125,1,2024,1,2,1,F,DU,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,0.0,0.0,DL,Delta Air Lines Inc.,D,DL,Delta Air Lines Inc.,3,31703,HPN,"White Plains, NY",New York,United States,New York,31703,JFK,"New York, NY",New York,United States,New York,6,622,1,2024,1,2,1,L,DU,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49.0,0.0,0.0,DL,Delta Air Lines Inc.,D,DL,Delta Air Lines Inc.,3,30721,BOS,"Boston, MA",Massachusetts,United States,Rhode Island,30721,PVD,"Providence, RI",Rhode Island,United States,Rhode Island,6,622,1,2024,3,7,1,L,DU,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,54.0,0.0,0.0,9K,Cape Air,D,9K,Cape Air,6,34992,STX,"Christiansted, VI",U.S. Virgin Islands,United States,Puerto Rico,35626,VQS,"Vieques, PR",Puerto Rico,United States,Puerto Rico,1,125,1,2024,2,4,1,F,DU,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80.0,0.0,0.0,DL,Delta Air Lines Inc.,D,DL,Delta Air Lines Inc.,3,33342,MKE,"Milwaukee, WI",Wisconsin,United States,Illinois,30977,MDW,"Chicago, IL",Illinois,United States,Illinois,6,622,1,2024,2,4,1,L,DU,0.0
